# SQL - Homework

## Please find the Entity Relationship Diagram of a simple database with two entities below: CLIENT and TRANSACTION

<img src="files/img/ERD_SQL_Homework.png">

#### Sample data of the CLIENT table:
| CLIENT_ID	| FIRST_NAME  | LAST_NAME |	CLIENT_SINCE |	CLIENT_EMAIL |
| --- | --- | --- | --- | --- |
|1|Alison|Blake|2017-01-01 00:00:00.000|Alison.Blake@gmail.com|
|2|Amanda|Bond|2017-05-01 00:00:00.000|Amanda.Bond@gmail.com|
|3|Amelia|Bower|2016-05-01 00:00:00.000|ABoweratgmail.com|
|4|Megan|Brown|2016-10-05 00:00:00.000|NULL|
|5|Melanie|Buckland|2018-01-05 00:00:00.000|Melanie.Buckland@gmail.com|

#### Sample data of the TRANSACTION table:
| TRAN_ID	| TRAN_DESCRIPTION  | DATE |	TRAN_AMOUNT |	CLIENT_ID |
| --- | --- | --- | --- | --- |
|1|ACH DEBIT|2018-01-01 00:00:00.000|-215.83|1|
|2|DEBIT CARD|2018-01-01 00:00:00.000|-209.16|1|
|3|ACH DEBIT|2018-01-02 00:00:00.000|-75|1|
|4|DEBIT CARD|2018-01-02 00:00:00.000|-63.92|1|
|5|INTERNET TRANSFER|2018-01-03 00:00:00.000|-25|1|
|6|NULL|2018-01-03 00:00:00.000|-25|1|
|7|DEBIT CARD LAS VEGAS|2018-01-03 00:00:00.000|-25|1|
|8|ACH DEBIT|2018-01-01 00:00:00.000|-9.99|2|
|9|DEBIT CARD|2018-01-01 00:00:00.000|-411.34|2|
|10|ACH DEBIT|2018-01-02 00:00:00.000|-65.99|2|

## Please write your answers to the following questions in a markdown cell:

- Note:
    - total time to complete the homework: about 30 minutes.

### Part A - Provide the SQL queries to answer the following questions (not the outputs):

The evaluation of your queries will emphasize logic more than syntax.

#### i) How many rows are there in the CLIENT table?

##### Question i - double click and write your answer below:
```*.sql
SELECT COUNT(*) AS NUMBER_OF_ROWS
FROM CLIENT
```

#### 1) How many transactions are there in the TRANSACTION table? (1 min)

##### Question 1 - double click and write your answer below:
```*.sql
SELECT COUNT(*) NUMBER_OF_TRANSACTIONS
FROM TRANSACTION;
```

#### 2) How many clients have been with the bank for at least 2 years (as of today, the day you do the homework)? (2 mins)

##### Question 2 - double click and write your answer below:
```*.sql
SELECT COUNT(*) TOTAL_CLIENT
FROM CLIENT
/*Filter condition for date at or less than 2 years stripping time since today is a date, not datetime*/
WHERE CAST(CLIENT_SINCE AS DATE) <= DATE_SUB(CURDATE(),INTERVAL 2 YEAR);
```

#### 3) Which client has the largest number of transactions? (2 mins)

##### Question 3 - double click and write your answer below:
```*.sql
SELECT CLIENT_ID, COUNT(*) TRAN_CT  
FROM TRANSACTION
/*Aggregates count by client*/
GROUP BY CLIENT_ID
/*Orders by column 2 - transaction count*/
ORDER BY 2 DESC
/*Selects top 1*/
LIMIT 1;
```

#### 4) Which clients have at least 5 ACH transactions ("ACH" is the type of the transaction, included in the description of the transaction)? (3 mins)

##### Question 4 - double click and write your answer below:
```*.sql
SELECT CLIENT_ID FROM TRANSACTION 
/*Ensures that the term ACH is included in the description.  This is not tolerant to words containing ACH and could throw false positives*/
WHERE TRAN_DESCRIPTION LIKE '%ACH%'
GROUP BY CLIENT_ID
/*Filters post agg for the count of each group to be at or above 5*/
HAVING COUNT(*) >= 5;
```

#### 5) Produce a list of all clients with their total number of transactions? (3 mins)

##### Question 5 - double click and write your answer below:
```*.sql
SELECT C.CLIENT_ID, count(TR.TRAN_ID) TRANSACTION_COUNT 
from CLIENT C 
/*Outer Join to allow for all client rows to show even when no transactions available*/
LEFT JOIN TRANSACTION TR ON C.CLIENT_ID = TR.CLIENT_ID
GROUP BY C.CLIENT_ID;
```

#### 6) Select all clients who have a valid email? (3 mins)

##### Question 6 - double click and write your answer below:
```*.sql
SELECT CLIENT_ID
FROM CLIENT 
/*Validates with regex to improve accuracy over comparing simply using a like search to ensure an @ and a . are in the email*/
WHERE CLIENT_EMAIL REGEXP '^[A-Z0-9._%-]+@[A-Z0-9.-]+[\.][A-Z]{2,4}$';
```

#### 7) Select the top 3 transactions for each client based on the \$ amount (absolute $ amount)? (5 mins)

[hint - TSQL](https://docs.microsoft.com/en-us/sql/t-sql/functions/row-number-transact-sql?view=sql-server-2017)

[hint - PL/SQL](https://docs.oracle.com/cd/B19306_01/server.102/b14200/functions137.htm)

[hint - MySQL](https://dev.mysql.com/doc/refman/8.0/en/window-functions-usage.html)

##### Question 7 - double click and write your answer below:
```*.sql
SELECT T_RNK.* 
FROM (
    SELECT *,
    /*Creates ranking based on absolute value of the transaction*/
    ROW_NUMBER() OVER (PARTITION BY CLIENT_ID ORDER BY ABS(TRAN_AMOUNT) DESC) RANKING
    from TRANSACTION
    ) T_RNK
/*filters for top 3*/
WHERE T_RNK.RANKING <=3
```

### Part B - For the following questions, provide an answer as plain english (not SQL queries):

#### 8) Do queries A and B produce the same result and why? (2 mins)
##### Query A:
```*.sql
SELECT COUNT(*) AS NUMBER_TRANSACTIONS
FROM [TRANSACTION]
;
``` 
##### Query B:

```*.sql
SELECT COUNT(TRAN_DESCRIPTION) AS NUMBER_TRANSACTIONS
FROM [TRANSACTION]
;
``` 

##### Question 8 - double click and write your answer below:
<div style="color: green;">

No.  The COUNT() function excludes NULL so if it's performed only on TRAN_DESCRIPTION then any null descriptions will be excluded whereas when it's performed on * it will count every row given the non-null PK.

</div>

#### 9) Context:

One of your collegue runs an ETL everyday (incremental load) in order to extract ACH transactions from the TRANSACTION table and use this piece of data as a feature for a machine learning model. The data pipeline is slow and the extraction of relevant transactions from the TRANSACTION table seems to be the bottleneck of this data pipeline. He asked the dba team (database administrator team) to create "releavant" indexes to smooth the process. The dba team replied that the TRANSACTION table already has an index on the primary key.

#### Can you explain what will be your approach to better diagnose the situation and offer suggestion(s) to improve the data pipeline? (Pros and Cons). (3 mins)

##### Question 9 - double click and write your answer below:
<div style="color: green;">

The first approach is to understand what query is being run in the ETL and what the non performant piece by doing visually evaluating the expression and running an explain plan or execution plan (depending on the db). Based on the structure above, I'll assume the that a wildcard search on TRAN_DESCRIPTION is being used to which I'd suggest one or more of the following
1. Indexed Search - adding an index to the column / columns being filtered on currently 
    1. Pro
        1. Quick to Add
        2. Can significantly improve query performance
    2. Con
        1. New Indexes can impact insert/update/delete performance
        2. String indexes are not going to improve performance only if the string filter starts with a wildcard (ex.'%XX')
<br><br>
2. Add more filters to the search and indexes - ensure that additional filters are added to narrow in the date range to only look at eligible transactions and add indexes for the new filters
    1. Pro
        1. Quick to Add
        2. Search will have less resource impact since a smaller subset of data is being processed
    2. Con
        1. May not be possible if updates to records outside of the date range need to be evaluated
        2. More indexes increase the impact on insert/update/delete
<br><br>
4. Creation and auto-population of a new Type table or column via - This solution involves adding a new column to capture the "Type" or creating a second table that includes the Type and PK (depending on which has less system impact).  The assumption is that the type is held in such a way and indexed where a non-wildcard query could be performed and that there are more than just ACH transactions so a scalable solution is needed 
    1. Trigger after insert/update/delete
        1. Pro
            1. No updates needed beyond the database
            2. An indexed, fully searchable column would significantly improve query performance
            3. The risk of table locking would drastically decrease
        2. Con
            1. The expertise needed to implement and maintain is a lot higher
            2. The insert/update/delete performance will be more significantly impacted with a trigger
            3. A table structure change could have wider impacts to other systems using the database 
            4. Time will need to be spent to backwards populate existing records
    2. Update the system writing the information
        1. Pro
            1. Insert/update/delete performance minimally impacted since the logic for the new identifier is performed before execution
            2. An indexed, fully searchable column would significantly improve query performance
            3. The risk of table locking would drastically decrease
            4. This helps ensure that the system updating the db and the db are more closely aligned on the datamodel
        2. Con
            1. Changing legacy systems can be costly and have lengthy timelines if the changes are even possible
            2. Grooming the different transaction types can prove lengthy 
            3. A table structure change could have wider impacts to other systems using the database 
            4. Time will need to be spent to backwards populate existing records

</div>

#### 10) How much time did you spend on this homework? Was it difficult or easy? (2 mins)

##### Question 10 - double click and write your answer below:
<div style="color: green;">

~40 min.  It was simple for the most part. A majority of the time was spent on formatting and annotating the answers. 
</div>